# Libraries

In [ ]:
pip install ortools



SyntaxError: invalid syntax (2381208173.py, line 1)

In [12]:
pip install typing-extensions



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
from IPython.display import display
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp


# Inputs

In [3]:
demand = [500, 600, 450, 550, 350]
production_per_worker_per_month_regular = 100
initial_workers = 10
desired_ending_inventory = [150, 180, 130, 160, 200]
final_required_inventory = 200
starting_inventory = 200
inventory_cost_per_liter = 0.05
layoff_cost_per_worker = 4
hiring_cost_per_worker = 1.5
regular_wage_per_worker = 4
overtime_wage_multiplier = 1.7
percentage_overtime = 0.3
overtime = [1, 1, 1, 1, 1]
months = len(demand)

# Main function

In [4]:
def aggregate_planning(demand, months, production_per_worker_per_month_regular, starting_inventory, desired_ending_inventory,
                        inventory_cost_per_liter, regular_wage_per_worker, hiring_cost_per_worker, layoff_cost_per_worker, initial_workers):

    # Create the mip solver with the SCIP backend.
    m = pywraplp.Solver.CreateSolver('SCIP')

    # Initialize variables
    workers = []  # Workers in each month
    hired_workers = []  # Workers hired in each month
    laid_off_workers = []  # Workers laid off in each month
    inventory = []  # Inventory levels for each month
    produced_units = [] # Production levels for each month

    for t in range(1,1+months):
        suffix = '_%s' % t
        hired_workers.append(m.IntVar(0,float('inf'),'H' + suffix))  # Workers hired in each month
        laid_off_workers.append(m.IntVar(0,float('inf'),'L' + suffix))  # Workers laid off in each month
        produced_units.append(m.NumVar(0,float('inf'),'p' + suffix))  # Production levels for each month

        workers.append(m.IntVar(0,float('inf'),'W' + suffix))  # Workers in each month
        inventory.append(m.NumVar(0,float('inf'),'I' + suffix))  # Inventory levels for each month



    # additional decision variables for use in the objecive
    objective = m.NumVar(0, float('inf'), 'total_cost')


    # constraints
    for t in range(months):
        m.Add(produced_units[t] == workers[t] * production_per_worker_per_month_regular)
        m.Add(inventory[t] >= desired_ending_inventory[t])

        #inventory and workers constraints
        #Constraints worker: The workforce in the current month = previous month's workforce + new hires - layoffs.
        if t > 0:
          m.Add(workers[t] == workers[t-1] + hired_workers[t] - laid_off_workers[t])
        else:
          m.Add(workers[t] == initial_workers + hired_workers[t] - laid_off_workers[t])

        #Constraints inventory : The inventory at the end of each month = previous inventory + production - demand.
        if t > 0:
          m.Add(inventory[t] == inventory[t-1] + produced_units[t] - demand[t])
        else:
          m.Add(inventory[t] == starting_inventory + produced_units[t] - demand[t])



    m.Add(inventory[months-1] >= desired_ending_inventory [t])

    # Objective function
    m.Add(objective == sum(
            workers[t] * regular_wage_per_worker +
            hired_workers[t] * hiring_cost_per_worker +
            laid_off_workers[t] * layoff_cost_per_worker +
            inventory[t] * inventory_cost_per_liter
            for t in range(months)
        )
    )


    print('Constraints =', m.constraints())

    # objective function
    m.Minimize(objective)

    status = m.Solve()
    # print(m.VerifySolution(0,True))

    if status == pywraplp.Solver.OPTIMAL:

        Results = {}
        for t in range(months):
            Results[t+1] = {'workers': workers[t].solution_value(),
                          'hired_workers': hired_workers[t].solution_value(),
                          'laied_off_workers': laid_off_workers[t].solution_value(),
                          'produced_units': round(produced_units[t].solution_value(),2),
                          'demand': round(demand[t]),
                          'final_inventory': round(inventory[t].solution_value(),2)
                          }
        df = pd.DataFrame(Results)
        display(df)
        print('The objective value is: ' + str(m.Objective().Value()))
        return df

    else:
        print('The problem does not have an optimal solution.')

In [6]:
# Call the function
df_results = aggregate_planning(demand, months, production_per_worker_per_month_regular, starting_inventory, desired_ending_inventory,
                        inventory_cost_per_liter, regular_wage_per_worker, hiring_cost_per_worker, layoff_cost_per_worker, initial_workers)

# Display results if a solution is found
if df_results is not None:
    print("\n✅ Optimal solution found!")
else:
    print("\n❌ No optimal solution found.")

Constraints = [<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x10bedbba0> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x10fee5020> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x111cd3b40> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11303daa0> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11303ef70> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11303f030> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x11303dad0> >, <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of

,1,2,3,4,5
workers,6.0,5.0,5.0,5.0,4.0
hired_workers,0.0,0.0,0.0,0.0,0.0
laied_off_workers,4.0,1.0,0.0,0.0,1.0
produced_units,600.0,500.0,500.0,500.0,400.0
demand,500.0,600.0,450.0,550.0,350.0
final_inventory,300.0,200.0,250.0,200.0,250.0


The objective value is: 184.00000000000003

✅ Optimal solution found!


In [7]:
def production(strategy, demand, desired_ending_inventory, starting_inventory, desired_month, overtime_months, percentage_overtime, inventory):

  if strategy == 1:  # Level Production Strategy
      required_production_per_month = (sum(demand) + sum(desired_ending_inventory) - starting_inventory)/len(demand)

  elif strategy == 2:  # Chase Strategy
        if desired_month == 1:
            required_production_per_month = demand[0] + desired_ending_inventory[0] - starting_inventory # Match demand exactly
        else:
            required_production_per_month = demand[desired_month - 1] + desired_ending_inventory[desired_month-1] - inventory [desired_month-2] # Match demand exactly

  elif strategy == 3:  # Mixed Strategy
        required_production_per_month = ((sum(demand) + sum(desired_ending_inventory) - starting_inventory)/(len(demand) + sum(o * percentage_overtime for o in overtime)))

  return required_production_per_month


In [13]:
from typing_extensions import final
def calculate_costs_and_inventory(t, produced_units, workers, regular_wage_per_worker, inventory, demand, inventory_cost_per_liter, total_wage_cost, total_inventory_cost):
    wage_cost = workers[t] * regular_wage_per_worker
    inventory[t + 1] = inventory[t] + produced_units - demand[t]
    inventory_cost = inventory[t + 1] * inventory_cost_per_liter

    total_wage_cost += wage_cost
    total_inventory_cost += inventory_cost

    return wage_cost, inventory_cost

def update_results(t, produced_units, wage_cost, inventory_cost, hiring_cost, layoff_cost, overtime_cost, workers, hired_workers, laid_off_workers,\
                   demand, inventory, results):
    total_monthly_cost = wage_cost + inventory_cost + hiring_cost + layoff_cost + overtime_cost

    results['Month'].append(t + 1) # it will add to the list created below
    results['Regular Workers'].append(workers[t])
    results['Hired Workers'].append(hired_workers[t])
    results['Laid-off Workers'].append(laid_off_workers[t])
    results['Units Produced'].append(produced_units)
    results['Demand'].append(demand[t])
    results['End Inventory'].append(inventory[t + 1])
    results['Wage Cost (in 1000s)'].append(wage_cost)
    results['Overtime Cost (in 1000s)'].append(overtime_cost)
    results['Hiring Cost (in 1000s)'].append(hiring_cost)
    results['Layoff Cost (in 1000s)'].append(layoff_cost)
    results['Inventory Cost (in 1000s)'].append(inventory_cost)
    results['Total Cost (in 1000s)'].append(total_monthly_cost)

def calculate_total_cost(strategy, demand, months, production_per_worker_per_month_regular, starting_inventory, desired_ending_inventory, \
                               inventory_cost_per_liter, regular_wage_per_worker, overtime_wage_multiplier, \
                                hiring_cost_per_worker, layoff_cost_per_worker, initial_workers, percentage_overtime):

    # Initialize variables
    inventory = np.zeros(months + 1)  # Inventory levels for each month, creates 13 items in the array
    inventory[0] = starting_inventory  # Set starting inventory, for the first item in the array in replace the 0 by a 50
    workers = np.zeros(months)  # Workers in each month, creates an array of 12 0s
    hired_workers = np.zeros(months)  # Workers hired in each month
    laid_off_workers = np.zeros(months)  # Workers laid off in each month

    # Tracking costs
    total_wage_cost = 0
    total_inventory_cost = 0
    total_overtime_cost = 0

    # Results table
    results = {
        'Month': [], 'Regular Workers': [], 'Hired Workers': [], 'Laid-off Workers': [],
        'Units Produced': [], 'Demand': [], 'End Inventory': [], 'Wage Cost (in 1000s)': [],
        'Overtime Cost (in 1000s)': [], 'Hiring Cost (in 1000s)': [], 'Layoff Cost (in 1000s)': [],
        'Inventory Cost (in 1000s)': [], 'Total Cost (in 1000s)': []
    }


    # Define overtime percentage for each month (e.g., 0.0 for no overtime, 0.2 for 20% overtime)
    overtime = [0.0] * months  # Initialize overtime to 0 for all months, list of 12 0s

    # Overtime is only used for Strategy 3 (Mixed Strategy)
    if strategy == 3:
        overtime = [1, 1, 1, 1, 1]  # Months with overtime for Mixed Strategy months:  all of them

    for desired_month in range(1,months+1):
        required_production_per_month = production(strategy,demand,desired_ending_inventory,starting_inventory,desired_month,overtime,percentage_overtime, inventory)
        workers[desired_month-1] = int(np.ceil(required_production_per_month/production_per_worker_per_month_regular))  # Round up to the nearest whole superior (np.ceil) number

        # Calculate hiring and layoffs based on previous workers
        t = desired_month - 1
        if t == 0:  # First month
            hired_workers[t] = max(0, workers[t] - initial_workers)  # Compare with initial workers
            laid_off_workers[t] = max(0, initial_workers - workers[t])  # Compare with initial workers
        else:
            hired_workers[t] = max(0, workers[t] - workers[t-1])  # Compare to previous month
            laid_off_workers[t] = max(0, workers[t-1] - workers[t])  # Compare to previous month

        # Update produced units with overtime percentage
        produced_units = workers[t] * production_per_worker_per_month_regular * (1 + overtime[t] * percentage_overtime)

        # Overtime is only used for Strategy 3 (Mixed Strategy)
        ##########################################################################
        # **Dynamic Overtime Calculation (Only When Needed)**

        if strategy == 3 :
            projected_inventory = inventory[t] + produced_units - demand[t]

            if projected_inventory < desired_ending_inventory[t]:
                deficit = abs(projected_inventory)  # Amount needed to bring inventory to 0

                # **Calculate the exact amount of overtime required**
                max_overtime_units = workers[t] * production_per_worker_per_month_regular * percentage_overtime  # Max 50% boost
                overtime_needed = min(1.0, deficit / max_overtime_units)  # Fraction of max overtime needed

                overtime[t] = overtime_needed  # Store overtime used for this month
                produced_units += overtime_needed * max_overtime_units  # Add overtime production

        # Calculate total hiring and layoff costs
        hiring_cost = hired_workers[t] * hiring_cost_per_worker
        layoff_cost = laid_off_workers[t] * layoff_cost_per_worker


        # Wage cost
        wage_cost, inventory_cost = calculate_costs_and_inventory(t, produced_units, workers, regular_wage_per_worker, inventory, demand, \
                                                                  inventory_cost_per_liter, total_wage_cost, total_inventory_cost)

        # Calculate overtime cost only for Mixed Strategy
        overtime_cost = 0
        if strategy == 3:
            overtime_cost = workers[t] * overtime[t] * percentage_overtime * regular_wage_per_worker * overtime_wage_multiplier
            total_overtime_cost += overtime_cost


        update_results(t, produced_units, wage_cost, inventory_cost, hiring_cost, layoff_cost, overtime_cost, workers, hired_workers, \
                       laid_off_workers, demand, inventory, results)

    return results

In [14]:
strategy = int(input("Enter the strategy number (1, 2, or 3): "))

results = calculate_total_cost(strategy, demand, months, production_per_worker_per_month_regular, starting_inventory, desired_ending_inventory, \
                               inventory_cost_per_liter, regular_wage_per_worker, overtime_wage_multiplier, \
                                hiring_cost_per_worker, layoff_cost_per_worker, initial_workers, percentage_overtime)

# Convert results to DataFrame for better readability
results_df = pd.DataFrame(results)
print("\nResults Summary:")
display(results_df)

# Total costs overview
total_cost = results_df['Total Cost (in 1000s)'].sum()
print(f"\nTotal Costs for the year (in 1000s): {total_cost}")



Results Summary:


,Month,Regular Workers,Hired Workers,Laid-off Workers,Units Produced,Demand,End Inventory,Wage Cost (in 1000s),Overtime Cost (in 1000s),Hiring Cost (in 1000s),Layoff Cost (in 1000s),Inventory Cost (in 1000s),Total Cost (in 1000s)
0,1,5.0,0.0,5.0,500.0,500,200.0,20.0,0,0.0,20.0,10.0,50.0
1,2,6.0,1.0,0.0,600.0,600,200.0,24.0,0,1.5,0.0,10.0,35.5
2,3,4.0,0.0,2.0,400.0,450,150.0,16.0,0,0.0,8.0,7.5,31.5
3,4,6.0,2.0,0.0,600.0,550,200.0,24.0,0,3.0,0.0,10.0,37.0
4,5,4.0,0.0,2.0,400.0,350,250.0,16.0,0,0.0,8.0,12.5,36.5



Total Costs for the year (in 1000s): 190.5
